# Distance Error Analysis
This notebook loads the raw CSV files, computes error/accuracy, and plots results.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

normal = pd.read_csv(r"/mnt/data/data/raw/normal_5_15_30cm.csv")
extremes = pd.read_csv(r"/mnt/data/data/raw/extremes_5_15_30cm.csv")
compensated = pd.read_csv(r"/mnt/data/data/raw/compensated_5_15_30cm.csv")

normal['scenario'] = 'normal'
extremes['scenario'] = 'extremes'
compensated['scenario'] = 'compensated'

df = pd.concat([normal, extremes, compensated], ignore_index=True)
df['error_cm'] = df['Measured_cm'] - df['Distance_cm']
df['abs_error_cm'] = df['error_cm'].abs()
df['reading_accuracy_percent'] = (1 - df['abs_error_cm'] / df['Distance_cm']).clip(lower=0) * 100
df

In [ ]:
# Measured vs True
plt.figure(figsize=(6,4))
for scenario, g in df.groupby('scenario'):
    plt.plot(g['Distance_cm'], g['Measured_cm'], marker='o', label=scenario)
plt.plot(sorted(df['Distance_cm'].unique()), sorted(df['Distance_cm'].unique()), linestyle='--', label='ideal')
plt.xlabel('True distance (cm)')
plt.ylabel('Measured (cm)')
plt.title('Measured vs True Distance')
plt.legend()
plt.show()

In [ ]:
# Absolute error per scenario
import numpy as np
plt.figure(figsize=(6,4))
targets = sorted(df['Distance_cm'].unique())
xs = np.arange(len(targets))
offset = 0.12
for i, (scenario, g) in enumerate(df.groupby('scenario')):
    errs = [g[g['Distance_cm']==t]['abs_error_cm'].values[0] for t in targets]
    x_shifted = xs + (i-1)*offset
    plt.plot(x_shifted, errs, marker='s', linestyle='-', label=scenario)
plt.xticks(xs, [f"{t} cm" for t in targets])
plt.xlabel('Target distance')
plt.ylabel('Absolute error (cm)')
plt.title('Absolute Error by Scenario')
plt.legend()
plt.show()